# Büyük Bir Metnin Tokenize Edilmesi ve Sıralanması

Şimdiye kadar bazı basit test cümleleri yazdık ve bir kelime dizini oluşturduk. Ardından cümeleler için diziler oluşturduk.

Şimdi, özellikle Amazon ve Yelp tarafından gelen incelemeler olmak üzere daha büyük bir metni simgeleştireceğiz (tokeniz edeceğiz) ve sıralayacağız.

## Veri Seti Hakkında

Ürün ve restoranların Amazon ve Yelp incelemelerini içeren bir veri seti kullanacaksınız. Bu veri seti orijinal olarak [Kaggle](https://www.kaggle.com/marklvl/sentiment-labelled-sentences-data-set)'dan alınmıştır.

Veri kümesi incelemeleri içerir ve her inceleme 0 (kötü) veya 1 (iyi) olarak etiketlenir. Ancak, bu alıştırmada, metni simgeleştirme ve sıralama alıştırması yapmak için etiketlere değil, yalnızca incelemelere odaklanalım.

### Bazı Örnek İyi Yorumlar

- This is hands down the best phone I've ever had.
- Four stars for the food & the guy in the blue shirt for his great vibe & still letting us in to eat !

### Bazı Kötü Yorumlar

- A lady at the table next to us found a live green caterpillar In her salad
- If you plan to use this in a car forget about it.

### Daha Fazla Örnek Görmek İçin

Veri setini indirmek ve yerel makinenizde incelemek istiyorsanız [buradan](https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P) indirebilirsiniz.


In [1]:
# İçeri Aktar: Tokenizer and pad_sequences
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# İçeri Aktar: numpy and pandas
import numpy as np
import pandas as pd

## Metnin İndirilmesi

Birleştirilmiş inceleme veri seti, Udacity'ye ait bir Google sürücüsüne kaydedildi. Oradan indirebilirsiniz.



In [2]:
path = tf.keras.utils.get_file('reviews.csv', 
                               'https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P')
print (path)

131072/127831 [==============================] - 0s 1us/step
/root/.keras/datasets/reviews.csv


## Veri Kümesinin Okunması

`.csv` dosyalarında her satır ayrı bir incelemedir.

`csv` dosyasında 2 sütun vardır:
- Metin
- Duygu (İyi veya kötü olma durumuna göre 0 veya 1)

In [3]:
# csv dosyasını okuyun
dataset = pd.read_csv(path)

# Veri kümesindeki ilk birkaç girişi gözden geçirin
dataset.head()

,Unnamed: 0,text,sentiment
0,0,So there is no way for me to plug it in here i...,0
1,1,Good case Excellent value.,1
2,2,Great for the jawbone.,1
3,3,Tied to charger for conversations lasting more...,0
4,4,The mic is great.,1


### CSV Dosyasından Değerlendirmelerin Alınması

Bu colab dosyasında etiketlerle ilgilenmeyeceğimiz için sadece yapılan yorumları alıyoruz.

In [4]:
# Metin sütunundan yorumları alın
reviews = dataset['text'].tolist()

## Metnin Simgeleştirilmesi (Tokenize Edilmesi)

Tokenize edici oluşturalım ve `OOV` belirtecini oluşturduğum tokenize ediciye verelim. Bu işlemlerden sonra kelime indekslerine bir göz atalım.

In [5]:
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(reviews)

word_index = tokenizer.word_index
print(len(word_index))
print(word_index)


3261
{'<OOV>': 1, 'the': 2, 'and': 3, 'i': 4, 'a': 5, 'it': 6, 'to': 7, 'is': 8, 'was': 9, 'this': 10, 'of': 11, 'not': 12, 'for': 13, 'my': 14, 'in': 15, 'with': 16, 'very': 17, 'good': 18, 'great': 19, 'phone': 20, 'that': 21, 'on': 22, 'have': 23, 'you': 24, 'food': 25, 'had': 26, 'place': 27, 'so': 28, 'but': 29, 'service': 30, 'are': 31, 'be': 32, 'we': 33, 'all': 34, 'as': 35, 'at': 36, 'like': 37, 'they': 38, 'time': 39, 'back': 40, 'one': 41, 'were': 42, 'quality': 43, 'would': 44, 'really': 45, 'here': 46, 'if': 47, 'from': 48, 'well': 49, 'your': 50, 'just': 51, 'product': 52, 'up': 53, 'best': 54, "don't": 55, 'no': 56, 'will': 57, 'an': 58, 'there': 59, 'go': 60, 'me': 61, 'has': 62, 'only': 63, 'also': 64, 'works': 65, "i've": 66, 'out': 67, 'headset': 68, 'nice': 69, 'ever': 70, 'battery': 71, "it's": 72, 'sound': 73, 'than': 74, 'use': 75, 'or': 76, 'when': 77, "i'm": 78, 'our': 79, 'get': 80, 'what': 81, 'their': 82, 'after': 83, 'love': 84, 'been': 85, 'did': 86, 'exce

## İncelemeler (Yorumlar) İçin Dizileri Oluşturalım

Her inceleme için bir dizi oluşturalım. En uzun incelemeyle eşleşmesi için maksimum uzunluğu ayarlayalım. En uzunu kadar uzun olmayan incelemeler için incelemenin sonuna dolgu sıfırları eklemeyi unutmayalım.

In [6]:
sequences = tokenizer.texts_to_sequences(reviews)
padded_sequences = pad_sequences(sequences, padding='post')

# Dolgulu dizileri içeren vektörün şekli nedir?
# Şekil (shape), dizilerin sayısını ve her birinin uzunluğunu gösterir.
print(padded_sequences.shape)

# İlk inceleme (yorum) nedir?
print (reviews[0])

# İlk inceleme için listeyi göster
print(padded_sequences[0])

# Diğer öğeler için inceleme ve dolgulu diziyi yazdırmayı deneyebilirsiniz.

(1992, 139)
So there is no way for me to plug it in here in the US unless I go by a converter.
[  28   59    8   56  142   13   61    7  269    6   15   46   15    2
  149  449    4   60  113    5 1429    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0]
